In [4]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.0


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt

In [6]:
airport = "RCTP"

train_df = pd.read_csv(airport + "/" + airport + "_train.csv")
test_df = pd.read_csv(airport + "/" + airport + "_test.csv")
train_df = train_df.reindex(np.random.permutation(train_df.index)) # shuffle the training set

In [10]:
print(train_df)

       windSpdKts  gustKts  windVrbDeg  visibilityMtrs   qnh  ws
22027          11        0           0            9999  1020   0
19032          16        0           0            9999  1018   0
2864           15        0           0            9999  1020   0
36896           9        0           0            9999  1002   0
21382           7        0           0            9999  1025   0
...           ...      ...         ...             ...   ...  ..
36444           3        0         130            9999  1004   0
1741           14        0           0            9999  1021   0
9598           14        0          70            9999  1022   0
34197          12        0           0            9999  1020   0
9833           10        0           0            1200  1014   0

[41070 rows x 6 columns]


In [11]:
train_df_norm = train_df.copy()
test_df_norm = test_df.copy()

# Scale down wind speed
train_df_norm["windSpdKts"] /= 50
test_df_norm["windSpdKts"] /= 50

# Scale down wind gust speed
train_df_norm["gustKts"] /= 50
test_df_norm["gustKts"] /= 50

# Scale down wind variability
train_df_norm["windVrbDeg"] /= 100
test_df_norm["windVrbDeg"] /= 100

# Scale down visibility
train_df_norm["visibilityMtrs"] /= 10000
test_df_norm["visibilityMtrs"] /= 10000

# Normalise, scale down QNH
train_df_norm["qnh"] -= 1013
train_df_norm["qnh"] /= 10
test_df_norm["qnh"] -= 1013
test_df_norm["qnh"] /= 10

print(train_df_norm)
print(test_df_norm)

       windSpdKts  gustKts  windVrbDeg  visibilityMtrs  qnh  ws
22027        0.22      0.0         0.0          0.9999  0.7   0
19032        0.32      0.0         0.0          0.9999  0.5   0
2864         0.30      0.0         0.0          0.9999  0.7   0
36896        0.18      0.0         0.0          0.9999 -1.1   0
21382        0.14      0.0         0.0          0.9999  1.2   0
...           ...      ...         ...             ...  ...  ..
36444        0.06      0.0         1.3          0.9999 -0.9   0
1741         0.28      0.0         0.0          0.9999  0.8   0
9598         0.28      0.0         0.7          0.9999  0.9   0
34197        0.24      0.0         0.0          0.9999  0.7   0
9833         0.20      0.0         0.0          0.1200  0.1   0

[41070 rows x 6 columns]
       windSpdKts  gustKts  windVrbDeg  visibilityMtrs  qnh  ws
0            0.14      0.0         0.0          0.9999  0.8   0
1            0.06      0.0         0.0          0.9999  0.7   0
2            0

In [13]:
feature_columns = []

windSpdKts = tf.feature_column.numeric_column("windSpdKts")
feature_columns.append(windSpdKts)

gustKts = tf.feature_column.numeric_column("gustKts")
feature_columns.append(gustKts)

windVrbDeg = tf.feature_column.numeric_column("windVrbDeg")
feature_columns.append(windVrbDeg)

vis_boundaries = list(np.arange(-1, 10000, 1000))
visibilityMtrs = tf.feature_column.numeric_column("visibilityMtrs")
visibilityMtrsBin = tf.feature_column.bucketized_column(visibilityMtrs, vis_boundaries)
feature_columns.append(visibilityMtrsBin)
# print(visibilityMtrsBin)

BucketizedColumn(source_column=NumericColumn(key='visibilityMtrs', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(-1, 999, 1999, 2999, 3999, 4999, 5999, 6999, 7999, 8999, 9999))


In [14]:
feature_layer = layers.DenseFeatures(feature_columns)

print(feature_layer(dict(train_df_norm)))

<tf.Tensor: shape=(41070, 15), dtype=float32, numpy=
array([[0.  , 0.  , 1.  , ..., 0.  , 0.22, 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.32, 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.3 , 0.  ],
       ...,
       [0.  , 0.  , 1.  , ..., 0.  , 0.28, 0.7 ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.24, 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.2 , 0.  ]], dtype=float32)>